In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
data=np.load("./data/PEMS04/pems04.npz")
data=data['data'][:,:,0]

In [ ]:
data.shape

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

X=data
y=data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create sequences for 12-to-12 prediction
def create_sequences(X, y, input_time_steps=12, output_time_steps=12):
    Xs, ys = [], []
    for i in range(len(X) - input_time_steps - output_time_steps + 1):
        Xs.append(X[i:(i + input_time_steps)])
        ys.append(y[(i + input_time_steps):(i + input_time_steps + output_time_steps)])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y)

# Split the dataset
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Dataset class
class MatchDatasetSeq(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Loaders
batch_size = 64
train_dataset_seq = MatchDatasetSeq(X_train_seq, y_train_seq)
test_dataset_seq = MatchDatasetSeq(X_test_seq, y_test_seq)

train_loader_seq = DataLoader(train_dataset_seq, batch_size=batch_size, shuffle=False)
test_loader_seq = DataLoader(test_dataset_seq, batch_size=batch_size, shuffle=False)

# LSTM model
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.attention = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output):
        # lstm_output: [batch_size, seq_len, hidden_dim]
        attention_weights = self.attention(lstm_output)
        attention_weights = torch.softmax(attention_weights, dim=1)
        # [batch_size, seq_len, 1]
        context_vector = attention_weights * lstm_output
        context_vector = torch.sum(context_vector, dim=1)
        # [batch_size, hidden_dim]
        return context_vector, attention_weights

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attention = Attention(hidden_dim)  # Add attention mechanism here
        self.fc = nn.Linear(hidden_dim, output_dim * 12)  # Adjusted for the output dimension

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        lstm_out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Apply attention
        context_vector, attention_weights = self.attention(lstm_out)
        
        out = self.fc(context_vector)
        out = out.view(x.size(0), 12, -1)  # Reshape to (batch_size, output_time_steps, output_dim)
        return out, attention_weights


# 初始化模型

In [ ]:
model = torch.load('model.pkl')

In [ ]:
model

In [ ]:
input_dim = X_train_seq.shape[2]
hidden_dim = 100
num_layers = 2
output_dim = y_train_seq.shape[2]

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    print(model)
    model.train()
    for inputs, labels in train_loader_seq:
        optimizer.zero_grad()
        outputs, attention_weights = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

torch.save(model, 'model.pkl')
# Evaluation
model.eval()
with torch.no_grad():
    all_preds = []
    all_labels = []
    for inputs, labels in test_loader_seq:
        outputs, attention_weights = model(inputs)
        all_preds.extend(outputs.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

mse = mean_squared_error(all_labels.reshape(-1), all_preds.reshape(-1))
rmse = sqrt(mse)
mae = mean_absolute_error(all_labels.reshape(-1), all_preds.reshape(-1))
r2 = r2_score(all_labels.reshape(-1), all_preds.reshape(-1))

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R^2: {r2}')